In [1]:
import os
os.environ["MKL_NUM_THREADS"] = "4" # export MKL_NUM_THREADS=6
import numpy as np
import logging
import sys,time
#sys.path.append('../')
os.chdir('../')
%load_ext line_profiler
from src.costs.full_shape_gradient import Full_shape_gradient

In [2]:
full_grad=Full_shape_gradient(path_config_file='config_file/config_full.ini')
init_param=full_grad.init_param

In [3]:
from src.costs.EM_cost import EM_cost_dask

In [4]:
%timeit full_grad.cost(init_param)

3.55 s ± 127 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit full_grad.shape_grad(init_param)

8.21 s ± 753 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%lprun -u 1 -f full_grad.shape_grad full_grad.shape_grad(init_param)

Timer unit: 1 s

Total time: 8.54321 s
File: /users/home/robin/stellacage_code/src/costs/full_shape_gradient.py
Function: shape_grad at line 47

Line #      Hits         Time  Per Hit   % Time  Line Contents
    47                                               def shape_grad(self,param_S_array):
    48         1          0.0      0.0      0.0          R=param_S_array[:len(self.m)]
    49         1          0.0      0.0      0.0          Z=param_S_array[len(self.m):]
    50         1          0.0      0.0      0.0          paramS=((self.m,self.n,R,Z))
    51         1          0.3      0.3      3.0          S=Surface_Fourier(paramS,(self.ntheta_coil,self.nzeta_coil),self.Np)
    52         1          2.3      2.3     27.0          theta_pertubation=S.get_theta_pertubation()
    53         1          4.5      4.5     52.1          shape_grad=(self.lst_cost[0]).shape_gradient(S,theta_pertubation)
    54         4          0.0      0.0      0.0          for elt in self.lst_cost[1:]:
    55

In [10]:
%lprun -u 1 -f EM_cost_dask full_grad.cost(init_param)

Timer unit: 1 s

Total time: 2.4813 s
File: /users/home/robin/stellacage_code/src/costs/EM_cost.py
Function: EM_cost_dask at line 84

Line #      Hits         Time  Per Hit   % Time  Line Contents
    84                                           def EM_cost_dask(config,S,Sp):
    85                                               #new version without Lagrange multipliers
    86         1          0.0      0.0      0.0      import dask.array as da
    87                                               #initilization of the parameters
    88         1          0.0      0.0      0.0      lamb = float(config['other']['lamb'])
    89         1          0.0      0.0      0.0      Np = int(config['geometry']['Np'])
    90         1          0.0      0.0      0.0      ntheta_plasma = int(config['geometry']['ntheta_plasma'])
    91         1          0.0      0.0      0.0      ntheta_coil   = int(config['geometry']['ntheta_coil'])
    92         1          0.0      0.0      0.0      nzeta_plasma = 

In [8]:
%timeit cost_surface(config)

2.5 s ± 82.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
shape_grad=Shape_gradient(config=config)
S_parametrization=shape_grad.S_parametrization

In [10]:
%lprun -u 1 -f shape_grad.compute_gradient_df shape_grad.compute_gradient_df(S_parametrization)

Timer unit: 1 s

Total time: 67.7384 s
File: main/shape_gradient.py
Function: compute_gradient_df at line 161

Line #      Hits         Time  Per Hit   % Time  Line Contents
   161                                               def compute_gradient_df(self,paramS):
   162                                                   #compute the shape gradient by a differentiation first method
   163         1          0.0      0.0      0.0          result={}
   164         1          0.3      0.3      0.5          S=Toroidal_surface(paramS,(self.ntheta_coil,self.nzeta_coil),self.Np)
   165         1          2.2      2.2      3.3          theta,dtildetheta,dtheta,dSdtheta=S.get_theta_pertubation()
   166         1          0.4      0.4      0.6          T=tools.get_tensor_distance(S,self.Sp,self.rot_tensor)
   167                                                   #LS=tools.compute_LS(T,self.matrixd_phi,S.jac_xyz,self.rot_tensor,self.Sp.n)
   168         1          0.1      0.1      0.1          Qj

In [11]:
%lprun -u 1 -f shape_grad.compute_gradient_of shape_grad.compute_gradient_of(S_parametrization)

Timer unit: 1 s

Total time: 6.16886 s
File: main/shape_gradient.py
Function: compute_gradient_of at line 59

Line #      Hits         Time  Per Hit   % Time  Line Contents
    59                                               def compute_gradient_of(self,paramS):
    60                                                   #compute the shape gradient by a optimization first method
    61         1          0.0      0.0      0.0          result={}
    62         1          0.3      0.3      5.3          S=Toroidal_surface(paramS,(self.ntheta_coil,self.nzeta_coil),self.Np)
    63                                                   # for latter, when we will implement GPU support
    64         1          0.0      0.0      0.0          f,f_np,get = lambda x : x,lambda x : x,lambda x : x
    65                                                   #tensors computations
    66         1          0.4      0.4      5.8          T=tools.get_tensor_distance(S,self.Sp,self.rot_tensor)
    67         1    

In [12]:
%timeit shape_grad.compute_gradient_of(S_parametrization)

KeyboardInterrupt: 